In [1]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [2]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 14s 0us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [4]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 2
layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = LeakyReLU(alpha=0.1)(layer2)
layer2 = MaxPooling2D(pool_size=(2, 2))(layer2)

# Layer 3
layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = LeakyReLU(alpha=0.1)(layer3)

# Layer 4
layer4 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = LeakyReLU(alpha=0.1)(layer4)

# Layer 5
layer5 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = LeakyReLU(alpha=0.1)(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

# Layer 6
layer6 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer5)
layer6 = BatchNormalization(name='norm_6')(layer6)
layer6 = LeakyReLU(alpha=0.1)(layer6)

# Layer 7
layer7 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = LeakyReLU(alpha=0.1)(layer7)

# Layer 8
layer8 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = LeakyReLU(alpha=0.1)(layer8)
layer8 = MaxPooling2D(pool_size=(2, 2))(layer8)

# Layer 9
layer9 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = LeakyReLU(alpha=0.1)(layer9)

# Layer 10
layer10 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = LeakyReLU(alpha=0.1)(layer10)

# Layer 11
layer11 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = LeakyReLU(alpha=0.1)(layer11)

# Layer 12
layer12 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer11)
layer12 = BatchNormalization(name='norm_12')(layer12)
layer12 = LeakyReLU(alpha=0.1)(layer12)

skip_connection = layer12

# Layer 13
layer13 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer12)
layer13 = BatchNormalization(name='norm_13')(layer13)
layer13 = LeakyReLU(alpha=0.1)(layer13)

layer13 = MaxPooling2D(pool_size=(2, 2))(layer13)

# Layer 14
layer14 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = LeakyReLU(alpha=0.1)(layer14)

# Layer 15
layer15 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = LeakyReLU(alpha=0.1)(layer15)

# Layer 16
layer16 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = LeakyReLU(alpha=0.1)(layer16)

# Layer 17
layer17 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = LeakyReLU(alpha=0.1)(layer17)

# Layer 18
layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = LeakyReLU(alpha=0.1)(layer18)

# Layer 19
layer19 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer18)
layer19 = BatchNormalization(name='norm_19')(layer19)
layer19 = LeakyReLU(alpha=0.1)(layer19)

# Layer 20
layer20 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer19)
layer20 = BatchNormalization(name='norm_20')(layer20)
layer20 = LeakyReLU(alpha=0.1)(layer20)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

layer21 = concatenate([skip_connection, layer20])

# Layer 22
layer22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer21)
layer22 = BatchNormalization(name='norm_22')(layer22)
layer22 = LeakyReLU(alpha=0.1)(layer22)

# Layer 23
layer23 = Flatten()(layer22)

output = Dense(num_classes, activation='softmax')(layer23)



Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
max_poolin

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 105s 2ms/step - loss: 1.4502 - acc: 0.4758 - val_loss: 1.6707 - val_acc: 0.4612
Epoch 2/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.9748 - acc: 0.6560 - val_loss: 1.5646 - val_acc: 0.5052
Epoch 3/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.7725 - acc: 0.7307 - val_loss: 1.1058 - val_acc: 0.6473
Epoch 4/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.6517 - acc: 0.7731 - val_loss: 0.9180 - val_acc: 0.7005
Epoch 5/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.5513 - acc: 0.8102 - val_loss: 1.1035 - val_acc: 0.6577
Epoch 6/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.4733 - acc: 0.8363 - val_loss: 1.0424 - val_acc: 0.6788
Epoch 7/50
50000/50000 [==============================] - 92s 2ms/step - loss: 

In [8]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 10s 965us/step
Test loss: 1.192112239408493
Test accuracy: 0.7781
Saved the model to disk


In [12]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

Saved the model to disk


In [13]:
from google.colab import files
files.download('Yolo_Basic_model2.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 50546, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [14]:
!ls -lrt

total 197600
drwxr-xr-x 1 root root      4096 Mar  8 17:26 sample_data
-rw-r--r-- 1 root root 202337408 Mar 16 12:24 Yolo_Basic_model2.h5
